In [1]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth

scope='user-library-read playlist-modify-public playlist-modify-private'

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="6584944ee0d9495480193d9997a9efb7",
                                               client_secret="ac4205f012b64179acd46c0fbdb33f36",
                                               redirect_uri="http://127.0.0.1:5000/redirect",
                                               scope=scope))

songs = []
pl = sp.current_user_playlists()['items'];
id = pl[0]['id']
playlist = sp.playlist_tracks(id)

tracks_data = []

for track in playlist['items']:
    
    #get the track popularity
    track_uri = track['track']['uri']
    track_info = sp.track(track_uri)
    track_popularity = track_info['popularity']
  
    track_data = {
            'artist_name': track['track']['artists'][0]['name'],
            'id': track['track']['id'],
            'track_pop': track_popularity,
            'danceability': None,
            'energy': None,
            'loudness': None,
            'speechiness': None,
            'acousticness': None,
            'instrumentalness': None,
            'liveness': None,
            'valence': None,
            'tempo': None
        }
    
    audio_features = sp.audio_features([track['track']['uri']])[0]
    if audio_features:
        track_data['danceability'] = audio_features['danceability']
        track_data['energy'] = audio_features['energy']
        track_data['loudness'] = audio_features['loudness']
        track_data['speechiness'] = audio_features['speechiness']
        track_data['acousticness'] = audio_features['acousticness']
        track_data['instrumentalness'] = audio_features['instrumentalness']
        track_data['liveness'] = audio_features['liveness']
        track_data['valence'] = audio_features['valence']
        track_data['tempo'] = audio_features['tempo']
        
    tracks_data.append(track_data)

songs_df = pd.DataFrame(tracks_data)
songs_df.drop_duplicates('id')

songs_df.head()


In [4]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
all_songs_df = pd.read_csv(r'C:\Users\alext\OneDrive\Desktop\FYP\data\data.csv')

#rearrange columns to match the user dataframe
desired_order = ['artists', 'id', 'track_pop','danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']
all_songs_df = all_songs_df[desired_order]

#cleaning the artists column - rename and only use FIRST artist in list
all_songs_df['artists'] = all_songs_df['artists'].str.replace(r"['\[\]]", '')
all_songs_df['artists'] = all_songs_df['artists'].str.split(',').str[0]
all_songs_df = all_songs_df.rename(columns={'artists': 'artist_name'})

pop = all_songs_df[["track_pop"]].reset_index(drop = True)
scaler = MinMaxScaler()
all_songs_df['track_pop'] = pd.DataFrame(scaler.fit_transform(pop), columns = pop.columns)
all_songs_df.head()


C:\Users\alext\AppData\Local\Temp\ipykernel_12620\2493739029.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  all_songs_df['artists'] = all_songs_df['artists'].str.replace(r"['\[\]]", '')


,artist_name,id,track_pop,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Sergei Rachmaninoff,4BJqT0PrAfrxzMOxytFOIz,0.04,0.279,0.211,-20.096,0.0366,0.982,0.878000,0.665,0.0594,80.954
1,Dennis Day,7xPhfUan2yNtyFG0cUWkt8,0.05,0.819,0.341,-12.441,0.4150,0.732,0.000000,0.160,0.9630,60.936
2,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,1o6I8BglA6ylDMrIELygv1,0.05,0.328,0.166,-14.850,0.0339,0.961,0.913000,0.101,0.0394,110.339
3,Frank Parker,3ftBPsC5vPBKxYSee08FDH,0.03,0.275,0.309,-9.316,0.0354,0.967,0.000028,0.381,0.1650,100.109
4,Phil Regan,4d6HGyGT8e121BsdKmw9v6,0.02,0.418,0.193,-10.096,0.0380,0.957,0.000002,0.229,0.2530,101.665
